# TD/ETD Comparison on "Random Walk"

This notebook contains some comparisons between TD and Emphatic TD on a simple problem under function approximation. 

We identify the solutions each algorithm converges to using the matrix operator equations for each algorithm, and compare them to the optimal approximation (as found by the least squares solution).

In [7]:
import numpy as np
import pandas as pd
from functools import reduce
from numpy import dot
from numpy.linalg import pinv

from features import *
from mdptools import *
from solvers import td_solution, etd_solution, exact_solution
from report import report

In [2]:
from IPython.display import display

# Random Walk

An environment which represents a random walk along a 1-D chain.

* Here, states are indexed from 0 to N-1, with states N and N-1 being the terminal states.
    * We refer to state `N-1` as the "rightmost" state, and state `N-2` as the "leftmost" state.
* The start state is in the middle of the chain, $s0 = \lfloor (N-2)/2 \rfloor$
* In terminal states, the feature vector $x(s)$ is the zero vector, the reward $r(s)$ is zero, and $\gamma(s) = \lambda(s) = i(s) = 0$.
* The environment is undiscounted, so $\gamma(s) = 1$ for $s$ non-terminal.
* For these experiments, $\lambda(s) = 0.5$ for $s$ non-terminal.
* Interest is constant and uniform for each state; $i(s) = 1$ for $s$ non-terminal.

In [3]:
def random_walk_matrix(n, p=0.5):
    """
    The transition matrix for a random walk with `n` states (including 
    two terminal states).
    """
    ret = np.zeros((n,n))
    # terminal state transitions
    ret[-2:, -2:] = np.eye(2) 
    # transient states that can terminate
    ret[0,-2] = p       # left side of chain
    ret[0,1] = (1-p)
    ret[-3,-4] = p      # right side of chain
    ret[-3,-1] = (1-p)
    # handle rest of transient states
    for i in range(1, n-3):
        ret[i][i-1] = p 
        ret[i][i+1] = (1-p)
    return ret

In [9]:
# Common parts of problem specification
num_states = 7
pleft = 0.5
pmat = random_walk_matrix(num_states, p=pleft)
s0 = int2basis(2, num_states)
states = state_vectors(pmat)
indices = state_indices(pmat)
terminals = [as_tuple(s) for s in find_terminals(pmat)]
gmfunc = Constant(1.0, terminals=terminals)
lmfunc = Constant(0.5, terminals=terminals)
ifunc = Constant(1.0, terminals=terminals)

In [10]:
display(pmat)

array([[ 0. ,  0.5,  0. ,  0. ,  0. ,  0.5,  0. ],
       [ 0.5,  0. ,  0.5,  0. ,  0. ,  0. ,  0. ],
       [ 0. ,  0.5,  0. ,  0.5,  0. ,  0. ,  0. ],
       [ 0. ,  0. ,  0.5,  0. ,  0.5,  0. ,  0. ],
       [ 0. ,  0. ,  0. ,  0.5,  0. ,  0. ,  0.5],
       [ 0. ,  0. ,  0. ,  0. ,  0. ,  1. ,  0. ],
       [ 0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  1. ]])

## Constant Feature, Unit Reward on Right-Termination

* reward for terminating in the rightmost state, `N-1`
    * r(s) = 1 for $(s, a, s') = (N-3, right, N-1)$
    * r(s) = 0 for all other transitions.
* x(s) = 1 for x non-terminal

In [11]:
rvec = np.zeros(num_states)
rvec[-3] = (1-pleft)
phi = Wrap(Bias(), terminals=terminals)

full_df = report(pmat, rvec, s0, phi, gmfunc, lmfunc, ifunc)
df = full_df[["weights", "MSE"]]

display(full_df)

Expected Reward:
[ 0.   0.   0.   0.   0.5  0.   0. ]
Feature Matrix:
[[ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 0.]
 [ 0.]]
True Values:
[ 0.16666667  0.33333333  0.5         0.66666667  0.83333333  0.          0.        ]
Emphasis As Good or Better?: True


,weights,MSE,values
Least-Squares,[0.5],0.055556,"[0.5, 0.5, 0.5, 0.5, 0.5, 0.0, 0.0]"
TD,[0.5],0.055556,"[0.5, 0.5, 0.5, 0.5, 0.5, 0.0, 0.0]"
ETD,[0.5],0.055556,"[0.5, 0.5, 0.5, 0.5, 0.5, 0.0, 0.0]"


In [12]:
print(df.to_latex())

\begin{tabular}{llr}
\toprule
{} & weights &       MSE \\
\midrule
Least-Squares &   [0.5] &  0.055556 \\
TD            &   [0.5] &  0.055556 \\
ETD           &   [0.5] &  0.055556 \\
\bottomrule
\end{tabular}



## Constant Reward, Constant Feature

* r(s) = 1 for s non-terminal
* x(s) = 1 for x non-terminal

This effectively calculates the expected number of steps to termination.

In [15]:
rfunc =  Constant(1.0, terminals=terminals)
rvec = np.array([rfunc(s) for s in states])
phi = Wrap(Bias(), terminals=terminals)

full_df = report(pmat, rvec, s0, phi, gmfunc, lmfunc, ifunc)
df = full_df[["weights", "MSE"]]

display(full_df)

Expected Reward:
[ 1.  1.  1.  1.  1.  0.  0.]
Feature Matrix:
[[ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 0.]
 [ 0.]]
True Values:
[ 5.  8.  9.  8.  5.  0.  0.]
Emphasis As Good or Better?: True


,weights,MSE,values
Least-Squares,[7.0],2.800000,"[7.0, 7.0, 7.0, 7.0, 7.0, 0.0, 0.0]"
TD,[8.36],4.649600,"[8.36, 8.36, 8.36, 8.36, 8.36, 0.0, 0.0]"
ETD,[7.66666666667],3.244444,"[7.66666666667, 7.66666666667, 7.66666666667, ..."


In [16]:
print(df.to_latex())

\begin{tabular}{llr}
\toprule
{} &          weights &       MSE \\
\midrule
Least-Squares &            [7.0] &  2.800000 \\
TD            &           [8.36] &  4.649600 \\
ETD           &  [7.66666666667] &  3.244444 \\
\bottomrule
\end{tabular}



## Increasing Reward, Constant Feature

* r(s) = s+1 for s non-terminal
* x(s) = 1 for x non-terminal

In [17]:
_rfunc = lambda x: basis2int(x) + 1
rfunc =  Parameter(_rfunc, terminals=terminals)
rvec = np.array([rfunc(s) for s in states])
phi = Wrap(Bias(), terminals=terminals)

full_df = report(pmat, rvec, s0, phi, gmfunc, lmfunc, ifunc)
df = full_df[["weights", "MSE"]]

display(full_df)

Expected Reward:
[ 1.  2.  3.  4.  5.  0.  0.]
Feature Matrix:
[[ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 0.]
 [ 0.]]
True Values:
[ 11.66666667  21.33333333  27.          26.66666667  18.33333333   0.           0.        ]
Emphasis As Good or Better?: True


,weights,MSE,values
Least-Squares,[21.0],32.488889,"[21.0, 21.0, 21.0, 21.0, 21.0, 0.0, 0.0]"
TD,[25.08],49.135289,"[25.08, 25.08, 25.08, 25.08, 25.08, 0.0, 0.0]"
ETD,[23.0],36.488889,"[23.0, 23.0, 23.0, 23.0, 23.0, 0.0, 0.0]"


In [18]:
print(df.to_latex())

\begin{tabular}{llr}
\toprule
{} &  weights &        MSE \\
\midrule
Least-Squares &   [21.0] &  32.488889 \\
TD            &  [25.08] &  49.135289 \\
ETD           &   [23.0] &  36.488889 \\
\bottomrule
\end{tabular}



## Decreasing Reward, Constant Feature

* r(s) = N-1-s for s non-terminal
* x(s) = 1 for x non-terminal

In [19]:
_rfunc = lambda x: (num_states - basis2int(x) - 1)
rfunc =  Parameter(_rfunc, terminals=terminals)
rvec = np.array([rfunc(s) for s in states])
phi = Wrap(Bias(), terminals=terminals)

full_df = report(pmat, rvec, s0, phi, gmfunc, lmfunc, ifunc)
df = full_df[["weights", "MSE"]]

display(full_df)

Expected Reward:
[ 6.  5.  4.  3.  2.  0.  0.]
Feature Matrix:
[[ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 0.]
 [ 0.]]
True Values:
[ 23.33333333  34.66666667  36.          29.33333333  16.66666667   0.           0.        ]
Emphasis As Good or Better?: True


,weights,MSE,values
Least-Squares,[28.0],52.088889,"[28.0, 28.0, 28.0, 28.0, 28.0, 0.0, 0.0]"
TD,[33.44],81.682489,"[33.44, 33.44, 33.44, 33.44, 33.44, 0.0, 0.0]"
ETD,[30.6666666667],59.200000,"[30.6666666667, 30.6666666667, 30.6666666667, ..."


In [20]:
print(df.to_latex())

\begin{tabular}{llr}
\toprule
{} &          weights &        MSE \\
\midrule
Least-Squares &           [28.0] &  52.088889 \\
TD            &          [33.44] &  81.682489 \\
ETD           &  [30.6666666667] &  59.200000 \\
\bottomrule
\end{tabular}



## Constant Reward, Increasing Feature

* r(s) = 1 for s non-terminal
* x(s) = s+1 for s non-terminal

In [21]:
rfunc =  Constant(1.0, terminals=terminals)
rvec = np.array([rfunc(s) for s in states])
phi = Wrap(Unary2Int(num_states), terminals=terminals)

full_df = report(pmat, rvec, s0, phi, gmfunc, lmfunc, ifunc)
df = full_df[["weights", "MSE"]]

display(full_df)

Expected Reward:
[ 1.  1.  1.  1.  1.  0.  0.]
Feature Matrix:
[[ 1.]
 [ 2.]
 [ 3.]
 [ 4.]
 [ 5.]
 [ 0.]
 [ 0.]]
True Values:
[ 5.  8.  9.  8.  5.  0.  0.]
Emphasis As Good or Better?: False


,weights,MSE,values
Least-Squares,[1.90909090909],11.709091,"[1.90909090909, 3.81818181818, 5.72727272727, ..."
TD,[1.90577507599],11.709212,"[1.90577507599, 3.81155015198, 5.71732522796, ..."
ETD,[1.72212978369],12.093590,"[1.72212978369, 3.44425956739, 5.16638935108, ..."


## Decreasing Reward, Increasing Feature

* r(s) = N-1-s for s non-terminal
* x(s) = s+1 for s non-terminal

In [22]:
_rfunc = lambda x: (num_states - basis2int(x) - 1)
rfunc =  Parameter(_rfunc, terminals=terminals)
rvec = np.array([rfunc(s) for s in states])
phi = Wrap(Unary2Int(num_states), terminals=terminals)

full_df = report(pmat, rvec, s0, phi, gmfunc, lmfunc, ifunc)
df = full_df[["weights", "MSE"]]

display(full_df)

Expected Reward:
[ 6.  5.  4.  3.  2.  0.  0.]
Feature Matrix:
[[ 1.]
 [ 2.]
 [ 3.]
 [ 4.]
 [ 5.]
 [ 0.]
 [ 0.]]
True Values:
[ 23.33333333  34.66666667  36.          29.33333333  16.66666667   0.           0.        ]
Emphasis As Good or Better?: False


,weights,MSE,values
Least-Squares,[7.29696969697],250.385455,"[7.29696969697, 14.5939393939, 21.8909090909, ..."
TD,[6.99088145897],251.416045,"[6.99088145897, 13.9817629179, 20.9726443769, ..."
ETD,[6.24958402662],262.452639,"[6.24958402662, 12.4991680532, 18.7487520799, ..."


In [23]:
print(df.to_latex())

\begin{tabular}{llr}
\toprule
{} &          weights &         MSE \\
\midrule
Least-Squares &  [7.29696969697] &  250.385455 \\
TD            &  [6.99088145897] &  251.416045 \\
ETD           &  [6.24958402662] &  262.452639 \\
\bottomrule
\end{tabular}

